In [1]:
from os import listdir
from os.path import isfile, join

mypath = "./datasets/images/"

file_names = [f for f in listdir(mypath) if isfile(join(mypath,f))]

print(str(len(file_names))+' images loaded')

3002 images loaded
